In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=5ba71ab9f8c1e86212c4ccea331f26a2740fb461e19de36b6acaf09e0c5e8fff
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [ ]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('weather DF').getOrCreate()

In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
df = spark.read.csv("drive/My Drive/Colab Notebooks/Weather Data in India from 1901 to 2017.csv",header=True,inferSchema=True)
df.show()
df.printSchema()

Mounted at /content/drive/
+---+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+
|_c0|YEAR|  JAN|  FEB|  MAR|  APR|  MAY|  JUN|  JUL|  AUG|  SEP|  OCT|  NOV|  DEC|
+---+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+
|  0|1901|17.99|19.43|23.49|26.41|28.28| 28.6|27.49|26.98|26.26|25.08|21.73|18.95|
|  1|1902| 19.0|20.39| 24.1|26.54|28.68|28.44|27.29|27.05|25.95|24.37|21.33|18.78|
|  2|1903|18.32|19.79|22.46|26.03|27.93|28.41|28.04|26.63|26.34|24.57|20.96|18.29|
|  3|1904|17.77|19.39|22.95|26.73|27.83|27.85|26.84|26.73|25.84|24.36|21.07|18.84|
|  4|1905| 17.4|17.79|21.78|24.84|28.32|28.69|27.67|27.47|26.29|26.16|22.07|18.71|
|  5|1906| 17.5|19.14|22.21|26.53|29.06|28.02|27.46|26.82|26.23|24.75|21.93|19.55|
|  6|1907|19.27|19.42|22.03|26.98|27.52|27.66|27.28|26.38|26.26|24.72|22.11|18.46|
|  7|1908|18.35|19.73|22.93|27.06|28.07|28.49|27.16|26.41|25.74|24.25|21.06|18.15|
|  8|1909|17.79|19.05| 23.4|25.76|27.97|27.67|26.56|26.43|25

In [ ]:
#1.Add column average temperature for each year

df.withColumn("Average", (col("JAN") + col("FEB") + col("MAR") + col("APR") + col("MAY") + col("JUN") + col("JUL") + col("AUG") + col("SEP") + col("OCT") + col("NOV") + col("DEC")) / 12)\
.show()

+---+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+------------------+
|_c0|YEAR|  JAN|  FEB|  MAR|  APR|  MAY|  JUN|  JUL|  AUG|  SEP|  OCT|  NOV|  DEC|           Average|
+---+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+------------------+
|  0|1901|17.99|19.43|23.49|26.41|28.28| 28.6|27.49|26.98|26.26|25.08|21.73|18.95|24.224166666666665|
|  1|1902| 19.0|20.39| 24.1|26.54|28.68|28.44|27.29|27.05|25.95|24.37|21.33|18.78|24.326666666666664|
|  2|1903|18.32|19.79|22.46|26.03|27.93|28.41|28.04|26.63|26.34|24.57|20.96|18.29|23.980833333333333|
|  3|1904|17.77|19.39|22.95|26.73|27.83|27.85|26.84|26.73|25.84|24.36|21.07|18.84|23.849999999999998|
|  4|1905| 17.4|17.79|21.78|24.84|28.32|28.69|27.67|27.47|26.29|26.16|22.07|18.71|           23.9325|
|  5|1906| 17.5|19.14|22.21|26.53|29.06|28.02|27.46|26.82|26.23|24.75|21.93|19.55|24.099999999999998|
|  6|1907|19.27|19.42|22.03|26.98|27.52|27.66|27.28|26.38|26.26|24.72|22.11|18.46|

In [ ]:
#2.Add row average temperature of each month

averages = [avg(col(month)).alias(month) for month in df.columns if month != "YEAR"]
average_row = df.groupBy().agg(*averages).collect()[0]
temp= Row(YEAR="Average", **average_row.asDict())
df = df.union(spark.createDataFrame([temp]))
df.show()

+---+------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+
|_c0|  YEAR|  JAN|  FEB|  MAR|  APR|  MAY|  JUN|  JUL|  AUG|  SEP|  OCT|  NOV|  DEC|
+---+------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+
|  0|1901.0|17.99|19.43|23.49|26.41|28.28| 28.6|27.49|26.98|26.26|25.08|21.73|18.95|
|  1|1902.0| 19.0|20.39| 24.1|26.54|28.68|28.44|27.29|27.05|25.95|24.37|21.33|18.78|
|  2|1903.0|18.32|19.79|22.46|26.03|27.93|28.41|28.04|26.63|26.34|24.57|20.96|18.29|
|  3|1904.0|17.77|19.39|22.95|26.73|27.83|27.85|26.84|26.73|25.84|24.36|21.07|18.84|
|  4|1905.0| 17.4|17.79|21.78|24.84|28.32|28.69|27.67|27.47|26.29|26.16|22.07|18.71|
|  5|1906.0| 17.5|19.14|22.21|26.53|29.06|28.02|27.46|26.82|26.23|24.75|21.93|19.55|
|  6|1907.0|19.27|19.42|22.03|26.98|27.52|27.66|27.28|26.38|26.26|24.72|22.11|18.46|
|  7|1908.0|18.35|19.73|22.93|27.06|28.07|28.49|27.16|26.41|25.74|24.25|21.06|18.15|
|  8|1909.0|17.79|19.05| 23.4|25.76|27.97|27.67|26.56|26.43|25.47

In [ ]:
#3.Add columns Min and Max for each year with corresponding values

min_max= df.groupBy("YEAR")\
    .agg(
        min("JAN").alias("Min_JAN"),
        min("FEB").alias("Min_FEB"),
        min("MAR").alias("Min_MAR"),
        min("APR").alias("Min_APR"),
        min("MAY").alias("Min_MAY"),
        min("JUN").alias("Min_JUN"),
        min("JUL").alias("Min_JUL"),
        min("AUG").alias("Min_AUG"),
        min("SEP").alias("Min_SEP"),
        min("OCT").alias("Min_OCT"),
        min("NOV").alias("Min_NOV"),
        min("DEC").alias("Min_DEC"),
        max("JAN").alias("Max_JAN"),
        max("FEB").alias("Max_FEB"),
        max("MAR").alias("Max_MAR"),
        max("APR").alias("Max_APR"),
        max("MAY").alias("Max_MAY"),
        max("JUN").alias("Max_JUN"),
        max("JUL").alias("Max_JUL"),
        max("AUG").alias("Max_AUG"),
        max("SEP").alias("Max_SEP"),
        max("OCT").alias("Max_OCT"),
        max("NOV").alias("Max_NOV"),
        max("DEC").alias("Max_DEC")
    )
dfmin_max = df.join(min_max, "YEAR")
dfmin_max.show()


+------+---+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+
|  YEAR|_c0|  JAN|  FEB|  MAR|  APR|  MAY|  JUN|  JUL|  AUG|  SEP|  OCT|  NOV|  DEC|Min_JAN|Min_FEB|Min_MAR|Min_APR|Min_MAY|Min_JUN|Min_JUL|Min_AUG|Min_SEP|Min_OCT|Min_NOV|Min_DEC|Max_JAN|Max_FEB|Max_MAR|Max_APR|Max_MAY|Max_JUN|Max_JUL|Max_AUG|Max_SEP|Max_OCT|Max_NOV|Max_DEC|
+------+---+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+
|1988.0| 87|18.58|20.83|23.51| 26.6| 28.6|28.37|27.12|26.91|26.48|24.79|21.63|19.64|  18.58|  20.83|  23.51|   26.6|   28.6|  28.37|  27.12|  26.91|  26.48|  24.79|  21.

In [ ]:
#4.Create dataframe called 'decade', storing average temperatures of each month in the respective decade. Decade 1910 is from 1901 to 1910
df = df.withColumn("decade", floor(col("YEAR") / 10) * 10)
decade = df.groupBy("decade").agg({month: "avg" for month in df.columns[2:]})
decade.show()



+------+------------------+------------------+-----------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+
|decade|          avg(APR)|          avg(AUG)|avg(decade)|          avg(SEP)|          avg(FEB)|          avg(JUN)|          avg(MAR)|          avg(OCT)|          avg(DEC)|          avg(JAN)|          avg(MAY)|          avg(JUL)|          avg(NOV)|
+------+------------------+------------------+-----------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+
|  1950|26.365999999999996|26.768999999999995|     1950.0|26.259999999999998|            20.018|28.253999999999998|            23.414|24.505000000000003|            19.195|18.451999999999998|28.302999999999997|            27.162|21.502999999999993|
|  1

In [ ]:
#5.


In [ ]:
#6.Which year is the coldest year? (Based on avg temp)

df = df.withColumn("Average", (col("JAN") + col("FEB") + col("MAR") + col("APR") + col("MAY") + col("JUN") + col("JUL") + col("AUG") + col("SEP") + col("OCT") + col("NOV") + col("DEC")) / 12)
hottest = df.orderBy(col("Average").desc()).first()
coldest = df.orderBy(col("Average")).first()
#hottest.show()
#coldest.show()
print("Hottest Year:", hottest.YEAR)
print("Coldest Year:", coldest.YEAR)

Hottest Year: 2016.0
Coldest Year: 1917.0


In [ ]:
#7.Find out which years recorded minimum and maximum temperatures. (Not based on avg temp)

min_max = df.select("YEAR", *[col(month).alias(f"{month}_Min") for month in df.columns[2:]],
                                    *[col(month).alias(f"{month}_Max") for month in df.columns[2:]])
min_max.show()

+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+----------+------------------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+----------+------------------+
|  YEAR|JAN_Min|FEB_Min|MAR_Min|APR_Min|MAY_Min|JUN_Min|JUL_Min|AUG_Min|SEP_Min|OCT_Min|NOV_Min|DEC_Min|decade_Min|       Average_Min|JAN_Max|FEB_Max|MAR_Max|APR_Max|MAY_Max|JUN_Max|JUL_Max|AUG_Max|SEP_Max|OCT_Max|NOV_Max|DEC_Max|decade_Max|       Average_Max|
+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+----------+------------------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+----------+------------------+
|1901.0|  17.99|  19.43|  23.49|  26.41|  28.28|   28.6|  27.49|  26.98|  26.26|  25.08|  21.73|  18.95|      1900|24.224166666666665|  17.99|  19.43|  23.49|  26.41|  28.28|   28.6|  27.49|  26.98|  26.26|  25.08|  2

In [ ]:
#8.How much raise is observed in min and max temperature for each month?
raisedf = df.select("YEAR","JAN","FEB","MAR","APR","MAY","JUN","JUL","AUG","SEP","OCT","NOV","DEC",
    (col("JAN") - col("FEB")).alias("JAN_Raise"),
    (col("FEB") - col("MAR")).alias("FEB_Raise"),
    (col("MAR") - col("APR")).alias("MAR_Raise"),
    (col("APR") - col("MAY")).alias("APR_Raise"),
    (col("MAY") - col("JUN")).alias("MAY_Raise"),
    (col("JUN") - col("JUL")).alias("JUN_Raise"),
    (col("JUL") - col("AUG")).alias("JUL_Raise"),
    (col("AUG") - col("SEP")).alias("AUG_Raise"),
    (col("SEP") - col("OCT")).alias("SEP_Raise"),
    (col("OCT") - col("NOV")).alias("OCT_Raise"),
    (col("NOV") - col("DEC")).alias("NOV_Raise")
)
raisedf.show()


+------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+--------------------+-------------------+-------------------+-------------------+--------------------+--------------------+-------------------+-------------------+------------------+------------------+------------------+
|  YEAR|  JAN|  FEB|  MAR|  APR|  MAY|  JUN|  JUL|  AUG|  SEP|  OCT|  NOV|  DEC|           JAN_Raise|          FEB_Raise|          MAR_Raise|          APR_Raise|           MAY_Raise|           JUN_Raise|          JUL_Raise|          AUG_Raise|         SEP_Raise|         OCT_Raise|         NOV_Raise|
+------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+--------------------+-------------------+-------------------+-------------------+--------------------+--------------------+-------------------+-------------------+------------------+------------------+------------------+
|1901.0|17.99|19.43|23.49|26.41|28.28| 28.6|27.49|26.98|26.26|25.08|21.73|18.95| -1.4400000000000